In [5]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

In [7]:
target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])

In [21]:
# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 70% training and 30% test

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

In [23]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [24]:
print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

[0]
[0]


In [25]:
# create a random generator
import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x



In [26]:
for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[4.351054989568278, 4.3570348555949145, 4.491423827688905, 0.34681966914231593]
[0]
[0]
[6.469401971694088, 2.176455313285955, 1.8868169414331404, 1.739448071664315]
[0]
[1]
[4.85299411574609, 2.959935273381828, 2.28430315179705, 2.3280056907555897]
[0]
[0]
[4.430649385878211, 4.386912083286276, 4.471077053736142, 0.7207619409225728]
[0]
[0]
[5.034024085905356, 2.140970292274957, 1.0985488595602984, 0.4681777518654344]
[0]
[0]


In [27]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)

In [28]:
bb.classify_example([3, 5, 4, 2])

0

In [29]:
from sygus_if import SyGuS_IF
X = [
    [1,2,1,1],
    [-1.1,1,1,42]
]
y = [
    1,
    0
]

In [30]:
sgf = SyGuS_IF()
# sgf.fit(X,y)
sgf.fit(X_test,y_test)
print(sgf.synthesized_function)

(define-fun func ((sepal_length Real) (sepal_width Real) (petal_length Real) (petal_width Real)) Real (ite (not (= petal_width 1)) (ite (> petal_length 1) (ite (>= sepal_length 5) (ite (< sepal_width 4) (ite (> sepal_length 5) (ite (>= petal_length 2) (ite (<= sepal_width 3) (ite (< sepal_width 3) (ite (>= sepal_length 6) (ite (>= petal_width 2) 0 (ite (= petal_length 4) (* (/ 1 4) petal_length) 0)) 1) 0) (- petal_width (/ 1 2))) 0) 0) 0) 0) (- petal_length 1)) petal_width))


In [16]:
y_pred_test = sgf.predict(X_test, y_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = sgf.predict(X_train, y_train)
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

Accuracy: 0.6666666666666666
Accuracy: 0.6666666666666666
